In [1]:
import azureml.core
from azureml.core import Workspace, Experiment, RunConfiguration
from azureml.core.runconfig import DataReferenceConfiguration
from azureml.core.script_run_config import ScriptRunConfig
import dotenv
%load_ext dotenv
%dotenv
import os

In [2]:
ws = Workspace.from_config()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [3]:
blob_datastore = ws.get_default_datastore()

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute_target import ComputeTargetException

aml_compute_target = "cpucluster"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("found existing compute target.")
except ComputeTargetException:
    print("creating new compute target")
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 2, 
                                                                max_nodes = 4)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Aml Compute attached")

found existing compute target.
Aml Compute attached


In [5]:
dataref_config = DataReferenceConfiguration(datastore_name=blob_datastore.name, 
                   path_on_datastore='bsdata', 
                   mode='mount', # download files from datastore to compute target
                   overwrite=True)

In [6]:
run_config = RunConfiguration(framework="PySpark")

# enable Docker 
run_config.environment.docker.enabled = True

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

# auto-prepare the Docker image when used for execution (if it is not already prepared)
run_config.auto_prepare_environment = True

run_config.environment.docker.base_image = "microsoft/mmlspark:0.12"

run_config.target = aml_compute
#run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['numpy'])
run_config.node_count = 2

run_config.environment.spark.repositories = []
run_config.environment.spark.packages = []

run_config.data_references = {blob_datastore.name: dataref_config}

In [7]:
script_run_config = ScriptRunConfig(source_directory=".",
                                    script="test_hanging.py",
                                    run_config=run_config,
                                    arguments=["--data_dir",str(blob_datastore.as_mount())])

run = Experiment(ws, 'script_run').submit(script_run_config)
run.wait_for_completion()

ActivityFailedException: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "No module named 'pandas'",
        "details": [],
        "debugInfo": {
            "type": "ModuleNotFoundError",
            "message": "No module named 'pandas'",
            "stackTrace": "  File \"/mnt/batch/tasks/shared/LS_root/jobs/antabsws/azureml/script_run_1558616784_8609d022/mounts/azureml_project_share/azureml/script_run_1558616784_8609d022/azureml-setup/context_manager_injector.py\", line 95, in execute_with_context\n    runpy.run_path(sys.argv[0], globals(), run_name=\"__main__\")\n  File \"/azureml-envs/azureml_2ffb57cb85eac5669f4d09127f2be92f/lib/python3.6/runpy.py\", line 263, in run_path\n    pkg_name=pkg_name, script_name=fname)\n  File \"/azureml-envs/azureml_2ffb57cb85eac5669f4d09127f2be92f/lib/python3.6/runpy.py\", line 96, in _run_module_code\n    mod_name, mod_spec, pkg_name, script_name)\n  File \"/azureml-envs/azureml_2ffb57cb85eac5669f4d09127f2be92f/li

In [ ]:
run